In [36]:
# ResNet
# residual block = path1(Conv2d + ReLU + Conv2d) + path2(identity) -> activation(ReLU)

from torchsummary import summary

import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.conv_path = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1)
        )
        self.skip_path = nn.Identity()
        self.out_act = nn.ReLU()
    def forward(self, x):
        x_conv = self.conv_path(x)
        x_skip = self.skip_path(x)
        x = x_conv + x_skip # concat이 아니라 값을 더한다.
        x = self.out_act(x)
        return x

block = ResidualBlock(in_channels=64, out_channels=64)
summary(block, (64, 56, 56))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]          36,928
              ReLU-2           [-1, 64, 56, 56]               0
            Conv2d-3           [-1, 64, 56, 56]          36,928
          Identity-4           [-1, 64, 56, 56]               0
              ReLU-5           [-1, 64, 56, 56]               0
Total params: 73,856
Trainable params: 73,856
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.77
Forward/backward pass size (MB): 7.66
Params size (MB): 0.28
Estimated Total Size (MB): 8.70
----------------------------------------------------------------


In [127]:
from torchsummary import summary
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.conv_path = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1, stride=1)
        )
        self.skip_path = nn.Identity()
        self.out_act = nn.ReLU()
    def forward(self, x):
        x_conv = self.conv_path(x)
        x_skip = self.skip_path(x)
        x = x_conv + x_skip
        x = self.out_act(x)
        return x

class ResidualBlockDown(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlockDown, self).__init__()
        self.conv_path = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, padding=1, stride=1)
        )
        self.skip_path = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, padding=0, stride=2)
        self.out_act = nn.ReLU()
    def forward(self, x):
        x_conv = self.conv_path(x)
        x_skip = self.skip_path(x)
        x = x_conv + x_skip
        x = self.out_act(x)
        return x

class ResNet34(nn.Module):
    def __init__(self):
        super(ResNet34, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, padding=3, stride=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, padding=1, stride=2)
        )
        self.conv2 = nn.Sequential(
            ResidualBlock(in_channels=64, out_channels=64),
            ResidualBlock(in_channels=64, out_channels=64),
            ResidualBlock(in_channels=64, out_channels=64)
        )
        self.conv3 = nn.Sequential(
            ResidualBlockDown(in_channels=64, out_channels=128),
            ResidualBlock(in_channels=128, out_channels=128),
            ResidualBlock(in_channels=128, out_channels=128),
            ResidualBlock(in_channels=128, out_channels=128)
        )
        self.conv4 = nn.Sequential(
            ResidualBlockDown(in_channels=128, out_channels=256),
            ResidualBlock(in_channels=256, out_channels=256),
            ResidualBlock(in_channels=256, out_channels=256),
            ResidualBlock(in_channels=256, out_channels=256),
            ResidualBlock(in_channels=256, out_channels=256),
            ResidualBlock(in_channels=256, out_channels=256)
        )
        self.conv5 = nn.Sequential(
            ResidualBlockDown(in_channels=256, out_channels=512),
            ResidualBlock(in_channels=512, out_channels=512),
            ResidualBlock(in_channels=512, out_channels=512),
        )
        self.avg_pool = nn.AvgPool2d(kernel_size=7, padding=0, stride=7)
        self.fc = nn.Linear(in_features=512, out_features=1024)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.avg_pool(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

model = ResNet34()
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
              ReLU-2         [-1, 64, 112, 112]               0
         MaxPool2d-3           [-1, 64, 56, 56]               0
            Conv2d-4           [-1, 64, 56, 56]          36,928
              ReLU-5           [-1, 64, 56, 56]               0
            Conv2d-6           [-1, 64, 56, 56]          36,928
          Identity-7           [-1, 64, 56, 56]               0
              ReLU-8           [-1, 64, 56, 56]               0
     ResidualBlock-9           [-1, 64, 56, 56]               0
           Conv2d-10           [-1, 64, 56, 56]          36,928
             ReLU-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,928
         Identity-13           [-1, 64, 56, 56]               0
             ReLU-14           [-1, 64,

In [129]:
from xmlrpc.client import Boolean
from torchsummary import summary
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv_path = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1, stride=stride),
            nn.ReLU(),
            nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, padding=1, stride=1)
        )
        if in_channels == out_channels and stride == 1:
            self.skip_path = nn.Identity()
        else:
            self.skip_path = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, padding=0, stride=stride)
        self.out_act = nn.ReLU()
    def forward(self, x):
        x_conv = self.conv_path(x)
        x_skip = self.skip_path(x)
        x = x_conv + x_skip
        x = self.out_act(x)
        return x

class ResNet34(nn.Module):
    def __init__(self):
        super(ResNet34, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, padding=3, stride=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, padding=1, stride=2)
        )
        self.conv2 = self._make_layers(in_channels=64, out_channels=64, n_blocks=3, downsample=False)
        self.conv3 = self._make_layers(in_channels=64, out_channels=128, n_blocks=4, downsample=True)
        self.conv4 = self._make_layers(in_channels=128, out_channels=256, n_blocks=6, downsample=True)
        self.conv5 = self._make_layers(in_channels=256, out_channels=512, n_blocks=3, downsample=True)
        self.avg_pool = nn.AvgPool2d(kernel_size=7, padding=0, stride=7)
        self.fc = nn.Linear(in_features=512, out_features=1024)

    def _make_layers(self, in_channels, out_channels, n_blocks, downsample=False):
        layers = []
        if downsample:
            layers.append(ResidualBlock(in_channels=in_channels, out_channels=out_channels, stride=2))
            in_channels = out_channels
        else: layers.append(ResidualBlock(in_channels=in_channels, out_channels=out_channels))

        for _ in range(n_blocks - 1):
            layers.append(ResidualBlock(in_channels=in_channels, out_channels=out_channels))
        
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.avg_pool(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

model = ResNet34()
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
              ReLU-2         [-1, 64, 112, 112]               0
         MaxPool2d-3           [-1, 64, 56, 56]               0
            Conv2d-4           [-1, 64, 56, 56]          36,928
              ReLU-5           [-1, 64, 56, 56]               0
            Conv2d-6           [-1, 64, 56, 56]          36,928
          Identity-7           [-1, 64, 56, 56]               0
              ReLU-8           [-1, 64, 56, 56]               0
     ResidualBlock-9           [-1, 64, 56, 56]               0
           Conv2d-10           [-1, 64, 56, 56]          36,928
             ReLU-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,928
         Identity-13           [-1, 64, 56, 56]               0
             ReLU-14           [-1, 64,

In [1]:
from xmlrpc.client import Boolean
from torchsummary import summary
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv_path = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1, stride=stride),
            nn.ReLU(),
            nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, padding=1, stride=1)
        )
        if in_channels == out_channels and stride == 1:
            self.skip_path = nn.Identity()
        else:
            self.skip_path = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, padding=0, stride=stride)
        self.out_act = nn.ReLU()
    def forward(self, x):
        x_conv = self.conv_path(x)
        x_skip = self.skip_path(x)
        x = x_conv + x_skip
        x = self.out_act(x)
        return x

class ResNet(nn.Module):
    def __init__(self, n_blocks_list):
        super(ResNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, padding=3, stride=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, padding=1, stride=2)
        )
        self.conv2 = self._make_layers(in_channels=64, out_channels=64, n_blocks=n_blocks_list[0], downsample=False)
        self.conv3 = self._make_layers(in_channels=64, out_channels=128, n_blocks=n_blocks_list[1], downsample=True)
        self.conv4 = self._make_layers(in_channels=128, out_channels=256, n_blocks=n_blocks_list[2], downsample=True)
        self.conv5 = self._make_layers(in_channels=256, out_channels=512, n_blocks=n_blocks_list[3], downsample=True)
        self.avg_pool = nn.AvgPool2d(kernel_size=7, padding=0, stride=7)
        self.fc = nn.Linear(in_features=512, out_features=1024)

    def _make_layers(self, in_channels, out_channels, n_blocks, downsample=False):
        layers = []
        if downsample:
            layers.append(ResidualBlock(in_channels=in_channels, out_channels=out_channels, stride=2))
            in_channels = out_channels
        else: layers.append(ResidualBlock(in_channels=in_channels, out_channels=out_channels))

        for _ in range(n_blocks - 1):
            layers.append(ResidualBlock(in_channels=in_channels, out_channels=out_channels))
        
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.avg_pool(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x
    
# 함수는 snake_case 이지만 return이 클래스이므로 camelCase 로 쓴다.
def ResNet18(): return ResNet(n_blocks_list=[2, 2, 2, 2])
def ResNet32(): return ResNet(n_blocks_list=[3, 4, 6, 3])
model = ResNet32()
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
              ReLU-2         [-1, 64, 112, 112]               0
         MaxPool2d-3           [-1, 64, 56, 56]               0
            Conv2d-4           [-1, 64, 56, 56]          36,928
              ReLU-5           [-1, 64, 56, 56]               0
            Conv2d-6           [-1, 64, 56, 56]          36,928
          Identity-7           [-1, 64, 56, 56]               0
              ReLU-8           [-1, 64, 56, 56]               0
     ResidualBlock-9           [-1, 64, 56, 56]               0
           Conv2d-10           [-1, 64, 56, 56]          36,928
             ReLU-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,928
         Identity-13           [-1, 64, 56, 56]               0
             ReLU-14           [-1, 64,